In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, RobustScaler
import warnings
import json
import os

warnings.filterwarnings('ignore')

## SECTION 1: DATA LOADING AND INITIAL SETUP

In [2]:
df_focal = pd.read_csv('../data/dataraw/data-1757985699237.csv')
df_competitors = pd.read_csv('../data/dataraw/data-1757985744315.csv')

df_focal['stay_date'] = pd.to_datetime(df_focal['stay_date'])
df_competitors['stay_date'] = pd.to_datetime(df_competitors['stay_date'])

print(f"Initial data shapes - Focal: {df_focal.shape}, Competitors: {df_competitors.shape}")

Initial data shapes - Focal: (5110, 4), Competitors: (1820, 5)


## SECTION 2: SYSTEMATIC MISSING VALUE TREATMENT

In [3]:
missing_focal = df_focal.isnull().sum().sum()
missing_comp = df_competitors.isnull().sum().sum()
print(f"Total missing values - Focal: {missing_focal}, Competitors: {missing_comp}")

if df_competitors['price'].isnull().sum() > 0:
    missing_by_date = df_competitors.groupby('stay_date')['price'].apply(lambda x: x.isnull().sum())
    high_missing_dates = missing_by_date[missing_by_date > 2].index
    print(f"Dates with high missing values (>2 hotels): {len(high_missing_dates)}")
    
    missing_by_hotel = df_competitors.groupby('hotel_id').agg({
        'price': ['count', lambda x: x.isnull().sum()]
    })
    missing_by_hotel.columns = ['total_obs', 'missing_count']
    missing_by_hotel['missing_rate'] = missing_by_hotel['missing_count'] / missing_by_hotel['total_obs']
    
    print("Missing data patterns by hotel:")
    print(missing_by_hotel[missing_by_hotel['missing_count'] > 0])
    
    df_competitors_clean = df_competitors.copy()
    df_competitors_clean = df_competitors_clean.sort_values(['hotel_id', 'stay_date'])
    df_competitors_clean['price'] = df_competitors_clean.groupby('hotel_id')['price'].fillna(method='ffill')
    
    median_imputer = df_competitors_clean.groupby('hotel_id')['price'].median()
    for hotel_id, median_price in median_imputer.items():
        mask = (df_competitors_clean['hotel_id'] == hotel_id) & (df_competitors_clean['price'].isnull())
        df_competitors_clean.loc[mask, 'price'] = median_price
    
    final_missing = df_competitors_clean['price'].isnull().sum()
    print(f"Remaining missing values after treatment: {final_missing}")
else:
    df_competitors_clean = df_competitors.copy()
    print("No missing price values detected")

print(f"Remaining missing values after treatment: {df_competitors_clean['price'].isnull().sum()}")

Total missing values - Focal: 0, Competitors: 30
Dates with high missing values (>2 hotels): 0
Missing data patterns by hotel:
                                                    total_obs  missing_count  \
hotel_id                                                                       
booking-us-aqua-pacific-monarch-USD                       348             16   
booking-us-courtyard-by-marriott-maui-kahului-a...        353             11   
booking-us-kohea-kai-resort-maui-USD                      362              2   
booking-us-ohana-waikiki-malia-USD                        363              1   

                                                    missing_rate  
hotel_id                                                          
booking-us-aqua-pacific-monarch-USD                     0.045977  
booking-us-courtyard-by-marriott-maui-kahului-a...      0.031161  
booking-us-kohea-kai-resort-maui-USD                    0.005525  
booking-us-ohana-waikiki-malia-USD                      0

## SECTION 3: PRICE VALIDATION AND OUTLIER TREATMENT

In [4]:
focal_p99 = df_focal['price'].quantile(0.99)
comp_p99 = df_competitors_clean['price'].quantile(0.99)
upper_bound = max(focal_p99, comp_p99) * 1.5

print(f"Price bounds - Upper: ${upper_bound:.0f} (1.5x 99th percentile)")

focal_invalid = (df_focal['price'] <= 0) | (df_focal['price'] > upper_bound)
comp_invalid = (df_competitors_clean['price'] <= 0) | (df_competitors_clean['price'] > upper_bound)

print(f"Invalid prices - Focal: {focal_invalid.sum()}, Competitors: {comp_invalid.sum()}")

def detect_outliers_iqr(df, price_col, group_col=None, multiplier=3.0):
    if group_col:
        outlier_mask = df.groupby(group_col)[price_col].apply(
            lambda x: (x < (x.quantile(0.25) - multiplier * (x.quantile(0.75) - x.quantile(0.25)))) |
                     (x > (x.quantile(0.75) + multiplier * (x.quantile(0.75) - x.quantile(0.25))))
        ).reset_index(level=0, drop=True)
    else:
        Q1, Q3 = df[price_col].quantile([0.25, 0.75])
        IQR = Q3 - Q1
        outlier_mask = (df[price_col] < (Q1 - multiplier * IQR)) | (df[price_col] > (Q3 + multiplier * IQR))
    return outlier_mask

focal_outliers = detect_outliers_iqr(df_focal, 'price', 'room_type', multiplier=3.0)
comp_outliers = detect_outliers_iqr(df_competitors_clean, 'price', 'hotel_id', multiplier=3.0)

print(f"Outliers - Focal: {focal_outliers.sum()}, Competitors: {comp_outliers.sum()}")

df_focal_clean = df_focal[~focal_outliers].copy()
df_competitors_final = df_competitors_clean[~comp_outliers].copy()

print(f"Clean shapes - Focal: {df_focal_clean.shape}, Competitors: {df_competitors_final.shape}")

Price bounds - Upper: $1498 (1.5x 99th percentile)
Invalid prices - Focal: 0, Competitors: 0
Outliers - Focal: 169, Competitors: 100
Clean shapes - Focal: (4941, 4), Competitors: (1720, 5)


In [5]:
df_focal_clean.columns

Index(['hotel_id', 'stay_date', 'room_type', 'price'], dtype='object')

## SECTION 4: BASE RATE EXTRACTION

In [6]:
df_focal_base = df_focal_clean.groupby('stay_date')['price'].min().reset_index()
df_focal_base.columns = ['stay_date', 'base_rate']

print(f"Base rate observations: {len(df_focal_base)}")
print(f"Base rate range: ${df_focal_base['base_rate'].min():.0f} - ${df_focal_base['base_rate'].max():.0f}")

Base rate observations: 365
Base rate range: $209 - $379


## SECTION 5: CROSS-HOTEL NORMALIZATION

In [7]:
focal_prices = df_focal_base['base_rate'].values
comp_prices = df_competitors_final['price'].values

focal_median = np.median(focal_prices)
focal_mad = np.median(np.abs(focal_prices - focal_median))
comp_median = np.median(comp_prices)
comp_mad = np.median(np.abs(comp_prices - comp_median))

print(f"Focal - Median: ${focal_median:.2f}, MAD: ${focal_mad:.2f}")
print(f"Competitor - Median: ${comp_median:.2f}, MAD: ${comp_mad:.2f}")

df_focal_base['base_rate_normalized'] = (focal_prices - focal_median) / focal_mad * comp_mad + comp_median
df_competitors_final['price_normalized'] = comp_prices

Focal - Median: $279.00, MAD: $20.00
Competitor - Median: $298.00, MAD: $36.57


## SECTION 6: FEATURE ENGINEERING FOR 2SRI

In [8]:
for df in [df_focal_base, df_competitors_final]:
    df['day_of_week'] = df['stay_date'].dt.dayofweek
    df['month'] = df['stay_date'].dt.month
    df['is_weekend'] = (df['day_of_week'] >= 5).astype(int)
    df['week_of_year'] = df['stay_date'].dt.isocalendar().week

df_focal_base = df_focal_base.sort_values(['stay_date'])
df_focal_base['base_rate_change'] = df_focal_base['base_rate'].diff()
df_focal_base['base_rate_change_pct'] = df_focal_base['base_rate'].pct_change()

df_competitors_final = df_competitors_final.sort_values(['hotel_id', 'stay_date'])
df_competitors_final['price_change'] = df_competitors_final.groupby('hotel_id')['price'].diff()
df_competitors_final['price_change_pct'] = df_competitors_final.groupby('hotel_id')['price'].pct_change()

for lag in [1, 2, 3]:
    df_focal_base[f'base_rate_lag_{lag}'] = df_focal_base['base_rate'].shift(lag)
    df_competitors_final[f'price_lag_{lag}'] = df_competitors_final.groupby('hotel_id')['price'].shift(lag)

if 'can_check_in' in df_competitors_final.columns:
    daily_availability = df_competitors_final.groupby('stay_date')['can_check_in'].mean().reset_index()
    daily_availability.columns = ['stay_date', 'market_availability']
    
    df_focal_base = df_focal_base.merge(daily_availability, on='stay_date', how='left')
    df_competitors_final = df_competitors_final.merge(daily_availability, on='stay_date', how='left')

## SECTION 7: DATA STRUCTURE PREPARATION FOR 2SRI

In [9]:
focal_daily = df_focal_base.groupby(['stay_date']).agg({
    'base_rate': 'mean',
    'base_rate_normalized': 'mean',
    'day_of_week': 'first',
    'month': 'first',
    'is_weekend': 'first'
}).reset_index()

comp_daily = df_competitors_final.groupby(['stay_date', 'hotel_id']).agg({
    'price': 'mean',
    'price_normalized': 'mean',
    'can_check_in': 'mean' if 'can_check_in' in df_competitors_final.columns else lambda x: 1,
    'day_of_week': 'first',
    'month': 'first',
    'is_weekend': 'first'
}).reset_index()

comp_price_pivot = comp_daily.pivot(index='stay_date', columns='hotel_id', values='price_normalized')
comp_availability_pivot = comp_daily.pivot(index='stay_date', columns='hotel_id', values='can_check_in' if 'can_check_in' in df_competitors_final.columns else 'hotel_id')

print(f"Initial data structures:")
print(f"  Focal daily: {focal_daily.shape}")
print(f"  Competitor price matrix: {comp_price_pivot.shape}")
print(f"  Missing values in competitor matrix: {comp_price_pivot.isnull().sum().sum()}")

focal_daily_indexed = focal_daily.set_index('stay_date')
comp_price_filled = comp_price_pivot.reindex(focal_daily_indexed.index)
comp_availability_filled = comp_availability_pivot.reindex(focal_daily_indexed.index)

print(f"After reindexing to focal date range:")
print(f"  Competitor matrix: {comp_price_filled.shape}")
print(f"  Missing values before filling: {comp_price_filled.isnull().sum().sum()}")

filling_stats = {}

for hotel_id in comp_price_filled.columns:
    original_nulls = comp_price_filled[hotel_id].isnull().sum()
    
    if original_nulls > 0:
        comp_price_filled[hotel_id] = comp_price_filled[hotel_id].fillna(method='ffill').fillna(method='bfill')
        
        if comp_price_filled[hotel_id].isnull().all():
            hotel_median = df_competitors_final[df_competitors_final['hotel_id'] == hotel_id]['price_normalized'].median()
            if pd.notna(hotel_median):
                comp_price_filled[hotel_id] = hotel_median
            else:
                market_median = df_competitors_final['price_normalized'].median()
                comp_price_filled[hotel_id] = market_median
                print(f"WARNING: {hotel_id} had no valid prices, used market median: ${market_median:.2f}")
        
        remaining_nulls = comp_price_filled[hotel_id].isnull().sum()
        if remaining_nulls > 0:
            comp_price_filled[hotel_id] = comp_price_filled[hotel_id].interpolate(method='linear')
            
        final_nulls = comp_price_filled[hotel_id].isnull().sum()
        if final_nulls > 0:
            hotel_median = comp_price_filled[hotel_id].median()
            comp_price_filled[hotel_id] = comp_price_filled[hotel_id].fillna(hotel_median)
    
    filled_nulls = original_nulls - comp_price_filled[hotel_id].isnull().sum()
    filling_stats[hotel_id] = {'original_nulls': original_nulls, 'filled': filled_nulls}

for hotel_id in comp_availability_filled.columns:
    if comp_availability_filled[hotel_id].isnull().any():
        comp_availability_filled[hotel_id] = comp_availability_filled[hotel_id].fillna(method='ffill').fillna(method='bfill').fillna(1)

print(f"Cross-hotel filling summary:")
for hotel_id, stats in filling_stats.items():
    if stats['filled'] > 0:
        print(f"  {hotel_id}: filled {stats['filled']} missing dates")

print(f"Final data quality:")
print(f"  Focal daily observations: {len(focal_daily)}")
print(f"  Competitor matrix shape: {comp_price_filled.shape}")
print(f"  Remaining missing values: {comp_price_filled.isnull().sum().sum()}")
print(f"  Complete date coverage: {comp_price_filled.shape[0] == len(focal_daily)}")

if comp_price_filled.isnull().sum().sum() > 0:
    print(f"ERROR: Still have {comp_price_filled.isnull().sum().sum()} missing values after filling!")
    for hotel_id in comp_price_filled.columns:
        nulls = comp_price_filled[hotel_id].isnull().sum()
        if nulls > 0:
            print(f"  {hotel_id}: {nulls} missing values")
else:
    print("SUCCESS: Complete data matrix achieved")

comp_price_pivot = comp_price_filled.reset_index()
comp_availability_pivot = comp_availability_filled.reset_index()

Initial data structures:
  Focal daily: (365, 6)
  Competitor price matrix: (364, 5)
  Missing values in competitor matrix: 100
After reindexing to focal date range:
  Competitor matrix: (365, 5)
  Missing values before filling: 105
Cross-hotel filling summary:
  booking-us-aqua-pacific-monarch-USD: filled 20 missing dates
  booking-us-castle-kamaole-sands-USD: filled 25 missing dates
  booking-us-courtyard-by-marriott-maui-kahului-airport-USD: filled 1 missing dates
  booking-us-kohea-kai-resort-maui-USD: filled 58 missing dates
  booking-us-ohana-waikiki-malia-USD: filled 1 missing dates
Final data quality:
  Focal daily observations: 365
  Competitor matrix shape: (365, 5)
  Remaining missing values: 0
  Complete date coverage: True
SUCCESS: Complete data matrix achieved


## SECTION 8: DATA QUALITY VALIDATION POST-PROCESSING

In [10]:
focal_missing_final = focal_daily.isnull().sum().sum()
comp_missing_final = comp_price_filled.isnull().sum().sum()
print(f"Final missing - Focal: {focal_missing_final}, Competitors: {comp_missing_final}")

if len(focal_daily) >= 10:
    focal_agg_daily = focal_daily.set_index('stay_date')['base_rate_normalized']
    comp_agg_daily = comp_price_filled.mean(axis=1)
    
    final_correlation = focal_agg_daily.corr(comp_agg_daily)
    print(f"Post-processing correlation: {final_correlation:.3f}")

total_focal_obs = len(focal_daily)
total_comp_obs = comp_price_filled.shape[0] * comp_price_filled.shape[1]
print(f"Final samples - Focal: {total_focal_obs}, Competitor matrix: {comp_price_filled.shape}")

if 'final_correlation' in locals() and abs(final_correlation) < 0.2:
    print("WARNING: Correlation significantly weakened, consider less aggressive cleaning")

Final missing - Focal: 0, Competitors: 0
Post-processing correlation: 0.362
Final samples - Focal: 365, Competitor matrix: (365, 5)


## SECTION 9: EXPORT PREPROCESSED DATA

In [12]:
os.makedirs('../data/dataprocessed', exist_ok=True)

df_focal_base.to_csv('../data/dataprocessed/focal_hotel_clean.csv', index=False)
df_competitors_final.to_csv('../data/dataprocessed/competitors_clean.csv', index=False)

focal_daily.to_csv('../data/dataprocessed/focal_daily_aggregated.csv', index=False)
comp_price_pivot.to_csv('../data/dataprocessed/competitor_price_matrix.csv', index=False)

if 'comp_availability_pivot' in locals():
    comp_availability_pivot.to_csv('../data/dataprocessed/competitor_availability_matrix.csv', index=False)

export_meta = {
    'focal_observations': int(total_focal_obs),
    'competitor_matrix_shape': [int(x) for x in comp_price_filled.shape],
    'date_range_days': int(len(focal_daily)),
    'date_start': focal_daily['stay_date'].min().isoformat(),
    'date_end': focal_daily['stay_date'].max().isoformat(),
    'final_correlation': float(final_correlation) if 'final_correlation' in locals() else None,
    'base_rate_range': [float(df_focal_base['base_rate'].min()), float(df_focal_base['base_rate'].max())],
    'outliers_removed': {'focal': int(focal_outliers.sum()), 'competitor': int(comp_outliers.sum())},
    'cross_hotel_fills': {k: int(v['filled']) for k, v in filling_stats.items() if v['filled'] > 0},
    'data_completeness': bool(comp_missing_final == 0 and focal_missing_final == 0)
}

with open('../data/dataprocessed/preprocessing_metadata.json', 'w') as f:
    json.dump(export_meta, f, indent=2)

print("Data preprocessing pipeline completed")
print("Files exported: focal_hotel_clean.csv, competitors_clean.csv, focal_daily_aggregated.csv, competitor_price_matrix.csv")

Data preprocessing pipeline completed
Files exported: focal_hotel_clean.csv, competitors_clean.csv, focal_daily_aggregated.csv, competitor_price_matrix.csv
